In [ ]:
import requests
import time

from datetime import datetime
from IPython.display import JSON

from unity_sds_client.unity import Unity
from unity_sds_client.unity_services import UnityServices
from unity_sds_client.resources.job_status import JobStatus
from unity_sds_client.unity import UnityEnvironments

In [ ]:
unity = Unity(UnityEnvironments.DEV)
unity.set_venue_id("")
process_service = unity.client(UnityServices.PROCESS_SERVICE)


In [3]:
process_service.endpoint = "http://unity-dev-httpd-alb-601260108.us-west-2.elb.amazonaws.com:8080/unity/dev/ogc/" # unity-venue-dev with 24.3 SPS deployment
#process_service.endpoint = "http://k8s-sps-ogcproce-b59e4702b8-2034669822.us-west-2.elb.amazonaws.com:5001" # one from Drew's notebook
#process_service.endpoint = "http://k8s-sps-ogcproce-927cdf8d63-717063809.us-west-2.elb.amazonaws.com:5001/" # EMIT endpoint

In [ ]:
process = process_service.get_process("cwl_dag")
print(process)


In [ ]:
data = {
  "inputs": {
    "cwl_workflow": "https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/main/demos/echo_message.cwl",
    "cwl_args": "{'message': 'Hello Godwin!'}",
    "request_memory": "8Gi",
    "request_cpu": "4",
    "request_storage": "10Gi"
  },
  "outputs": {
    "result": {
      "transmissionMode": "reference"
    }
  }
}
submitted_job = process.execute(data)

In [ ]:
data = {
  "inputs": {
    "cwl_workflow": "https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/main/demos/echo_message.cwl",
    "cwl_args": "{'message': 'Hello Brian!'}",
    "request_memory": "8Gi",
    "request_cpu": "4",
    "request_storage": "10Gi"
  },
  "outputs": {
    "result": {
      "transmissionMode": "reference"
    }
  }
}
submitted_job = process.execute(data)

In [ ]:
import time

status = submitted_job.get_status().status
while status in [JobStatus.ACCEPTED, JobStatus.RUNNING]:
    print("job {} is not complete. Currently in state {} ".format(submitted_job.id, submitted_job.get_status().status))
    time.sleep(5)
    status = submitted_job.get_status().status

print("Job {} completed with status {}".format(submitted_job.id, str(submitted_job.get_status().status)))


In [ ]:
data = {
  "inputs": {
    "cwl_workflow": "http://awslbdockstorestack-lb-1429770210.us-west-2.elb.amazonaws.com:9998/api/ga4gh/trs/v2/tools/%23workflow%2Fdockstore.org%2FGodwinShen%2Femit-ghg/versions/9/plain-CWL/descriptor/workflow.cwl",
    "cwl_args": "https://raw.githubusercontent.com/GodwinShen/emit-ghg/main/test/emit-ghg-test.json",
    "request_memory": "8Gi",
    "request_cpu": "4",
    "request_storage": "10Gi"
  },
  "outputs": {
    "result": {
      "transmissionMode": "reference"
    }
  }
}
submitted_job = process.execute(data)